In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv',
    parse_dates=['date']
)
item_names = sorted(df['type_name'].unique())
item_dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Type or select an item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)
ma_slider = widgets.IntSlider(
    value=7, min=2, max=60, step=1, description='MA Window', continuous_update=False
)
output = widgets.Output()
display(item_dropdown, ma_slider, output)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, options=(' Tyrant Blue Sa…

IntSlider(value=7, continuous_update=False, description='MA Window', max=60, min=2)

Output()

In [4]:
def plot_ma_crossover(item, ma_window):
    with output:
        clear_output(wait=True)
        if not item or item not in df['type_name'].values:
            print("Please select a valid item.")
            return
        item_df = df[df['type_name'] == item].sort_values('date').copy()
        item_df['MA'] = item_df['average'].rolling(window=ma_window, min_periods=1).mean()

        # Buy (price crosses above MA), Sell (below MA)
        buy_signals = (item_df['average'] > item_df['MA']) & (item_df['average'].shift(1) <= item_df['MA'].shift(1))
        sell_signals = (item_df['average'] < item_df['MA']) & (item_df['average'].shift(1) >= item_df['MA'].shift(1))

        print(f"MA window: {ma_window} days | Buy signals: {buy_signals.sum()} | Sell signals: {sell_signals.sum()}")
        plt.figure(figsize=(12,6))
        plt.plot(item_df['date'], item_df['average'], label='Price', color='blue')
        plt.plot(item_df['date'], item_df['MA'], label=f'{ma_window}-Day MA', color='orange')
        plt.scatter(item_df.loc[buy_signals, 'date'], item_df.loc[buy_signals, 'average'],
                    label='Buy', marker='^', color='green', s=100)
        plt.scatter(item_df.loc[sell_signals, 'date'], item_df.loc[sell_signals, 'average'],
                    label='Sell', marker='v', color='red', s=100)
        plt.title(f"{item} — Price & {ma_window}-Day MA\nBuy/Sell Signals Shown")
        plt.xlabel("Date")
        plt.ylabel("Average Price (ISK)")
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()

widgets.interactive_output(
    plot_ma_crossover,
    {'item': item_dropdown, 'ma_window': ma_slider}
)


Output()